<a href="https://colab.research.google.com/github/ElizavetaNosova/Quest-generation-bot/blob/main/generate_and_analize_quest_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cpufeature
!pip install triton==0.2.3
!DS_BUILD_CPU_ADAM=1 DS_BUILD_SPARSE_ATTN=1 pip install deepspeed==0.3.7
!pip install transformers==3.5.1

  Using cached https://files.pythonhosted.org/packages/a9/3b/7b9c8619e9eb4e0eb3cf9e01dccd1b85161f5d9e452d7fbf3aca23a62675/triton-0.2.3.tar.gz
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.
  Using cached https://files.pythonhosted.org/packages/1f/f6/4de24b5790621e9eb787b7e4d90a57075ebbb85e81100a0dc8c50fdba8ba/deepspeed-0.3.7.tar.gz
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


Тетрадка с генерацией примеров квестов с разными параметрами генерации с использованием фрагментов не использованных при обучении текстовых квестов и подсчетом метрик


In [ ]:

!pip install torch.optim

ERROR: Could not find a version that satisfies the requirement torch.optim (from versions: none)
ERROR: No matching distribution found for torch.optim


In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import random
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re 
from tqdm import tqdm
import json
import pandas as pd
import statistics

In [ ]:
os.chdir('gdrive/MyDrive')

In [ ]:
test_file = os.listdir('Quest_data/questbook_fragments_json')[0]

In [ ]:
with open(os.path.join('Quest_data/questbook_fragments_json', test_file)) as f:
  test_data = json.load(f)

In [ ]:
QUEST_FRAGMENTS_DIRECTORY = 'Quest_data/questbook_fragments_json'
quest_starts = []
for file in os.listdir(QUEST_FRAGMENTS_DIRECTORY):
    with open(os.path.join(QUEST_FRAGMENTS_DIRECTORY, file)) as f:
        fragments = json.load(f)
       # print(len(fragments))
       # print(type(fragments))
    fragments_ids = list(fragments.keys())
    if fragments_ids:
        digit_ids = [fragment_id for fragment_id in fragments_ids if fragment_id.isdigit()]
        if digit_ids:
            chosen_id = sorted(digit_ids, key=lambda x: int(x))[0]
        else:
            chosen_id = random.choice(fragments_ids)
        quest_starts.append(fragments[chosen_id])

In [ ]:
len(quest_starts)

136

In [ ]:
with open('first_quest_fragments4test.json', 'w', encoding='utf-8') as f:
    json.dump(quest_starts, f)

In [ ]:
with open('first_quest_fragments4test.json', encoding='utf-8') as f:
    quest_starts = json.load(f)

In [ ]:
TOKENIZER_PATH = "home/jovyan/devices/quests/hf"
MODEL_PATH = "home/jovyan/devices/quests/hf"

gpt_model = GPT2LMHeadModel.from_pretrained(MODEL_PATH)
tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)

In [ ]:
gpt_model.cuda()

In [ ]:
class GptGenerator:
    def __init__(self, gpt_model, tokenizer, max_pretext_length = 2048):
        self.gpt_model = gpt_model
        self.tokenizer = tokenizer
        self.__max_pretext_length = max_pretext_length
        self.tokenizer.add_special_tokens({"bos_token": "<s>"})
        self.tokenizer.add_special_tokens({"eos_token": "</s>"})      
        
        self.default_greedy_temperature = 1
        self.default_not_greedy_temperature = 1.5
    
     
    def get_next_steps(self, text, max_length=200, greedy=False, temperature=None, num_beams=None):
        pretext_length = len(text)
        input_idx = self.tokenizer.encode(text, return_tensors='pt').cuda()
        if input_idx.shape[1] > self.__max_pretext_length-max_length:
            input_idx = input_idx[:, -(self.__max_pretext_length-max_length):]
            used_pretext = tokenizer.decode(input_idx[0])
            pretext_length = len(used_pretext)
        if temperature == None:
            if greedy:
                temperature = self.default_greedy_temperature
            else:
                temperature = self.default_not_greedy_temperature
        if greedy:
             encoded_new_text = self.gpt_model.generate(input_idx, max_length=self.__max_pretext_length, temperature=temperature, repetition_penalty=5.0, top_k=5, top_p=0.95)[0]
        else:
           # encoded_new_text = self.gpt_model.generate(input_idx, max_length=pretext_length+max_length, num_beams=num_beams, temperature=temperature
           encoded_new_text = self.gpt_model.generate(input_idx, max_length=self.__max_pretext_length, num_beams=num_beams, temperature=temperature, do_sample=True)[0]         
        new_text = self.tokenizer.decode(encoded_new_text)
        return new_text[pretext_length:]

In [ ]:
gpt_generator = GptGenerator(gpt_model, tokenizer)

In [ ]:
current_test_dir = os.path.join('test_generation', '3_beams_button_input')

In [ ]:
os.mkdir(current_test_dir)

In [ ]:
MAX_TEXT_LENGTH = 250000
MAX_FRAGMENTS = 75

In [ ]:
start_from = 135
for i, fragment in tqdm(enumerate(quest_starts[start_from:])):
    print(i+start_from)
    text = '\n'.join(['BOQ', 'BOF', fragment, 'BUTTON'])
    prev_text_len = 0
    while 'EOQ' not in text and len(text) < MAX_TEXT_LENGTH and len(re.findall('BOF', text)) < MAX_FRAGMENTS and len(text) > prev_text_len:
        prev_text_len = len(text)
        text += gpt_generator.get_next_steps(text, num_beams=3)
    with open(os.path.join(current_test_dir, str(i+start_from)+'.txt'), 'w', encoding='utf-8') as f:
        f.write(text)
           

0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


135


1it [01:39, 99.84s/it]


In [ ]:
def count_mean_fragment_len(quest):
    parts = quest.split('BOF')
    if len(parts) > 2:
        # Part 0 is a technical part before the first fragment, part 1 is a fragment which is not generated
        generated_parts = parts[2:]
        fragments_without_buttons = [part.split('BUTTON')[0] for part in parts]
        return statistics.mean([len(fragment) for fragment in fragments_without_buttons])

In [ ]:
len(os.listdir(current_test_dir))

135

In [ ]:
finished_quests_metrics = pd.DataFrame(columns = ['quest_file', 'total_len', 'num_fragments', 'num_buttons', 'mean_fragment_len'])

for file_name in os.listdir(current_test_dir):
    file_path = os.path.join(current_test_dir, file_name)
    with open(file_path, encoding='utf-8') as f:
        text = f.read()
    if 'EOQ' in text:
        quest = text.split('EOQ')[0]
        total_len = len(quest)
        num_fragments = len(re.findall('BOF', quest))
        num_buttons = len(re.findall('BUTTON', quest))
        mean_fragment_len = count_mean_fragment_len(quest)
        finished_quests_metrics.loc[len(finished_quests_metrics)] = [file_name, total_len, num_fragments, num_buttons, mean_fragment_len]


In [ ]:
#количество законченных квестов
len(finished_quests_metrics)

120

In [ ]:
#доля законченных квестов
120/135

0.8888888888888888

In [ ]:
def count_mean_num_buttons_for_finished_quest(num_fragments, num_buttons):
    return num_buttons/(num_fragments-1) if num_fragments > 1 else 0

In [ ]:
for column in ['total_len', 'num_buttons', 'num_fragments', 'mean_fragment_len']:
    finished_quests_metrics[column] = pd.to_numeric(finished_quests_metrics[column])

In [ ]:
finished_quests_metrics['buttons_for_fragment'] = finished_quests_metrics.apply(lambda x: count_mean_num_buttons_for_finished_quest(x.num_fragments, x.num_buttons), axis=1)

In [ ]:
finished_quests_metrics.loc[:, ['total_len', 'num_fragments', 'buttons_for_fragment', 'mean_fragment_len']].describe()

,total_len,num_fragments,buttons_for_fragment,mean_fragment_len
count,120.000000,120.000000,120.000000,119.000000
mean,17224.191667,20.875000,1.734796,774.461946
std,19552.427779,16.437589,0.622334,990.192966
min,147.000000,1.000000,0.000000,150.909091
25%,5431.250000,9.000000,1.495690,383.331169
50%,9847.000000,16.000000,1.629573,580.609375
75%,20334.000000,29.000000,1.857143,763.870588
max,113796.000000,80.000000,6.000000,7507.066667


In [ ]:
source_data_directory = os.path.join('Quest_data', 'quest_book_online_random_paths_text')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize
from collections import defaultdict
from tqdm import tqdm

In [ ]:
source_sents = defaultdict(set)
for quest_directory in tqdm(os.listdir(source_data_directory)):
    quest_directory_path = os.path.join(source_data_directory, quest_directory)
    if os.path.isdir(quest_directory_path):
        for text_option_file in os.listdir(quest_directory_path):
            with open(os.path.join(source_data_directory, quest_directory, text_option_file), encoding='utf-8') as f:
                text_option = f.read()
            text_without_tags = re.sub('BO[QF]|EOQ|BUTTON|INPUT', '', text_option)
            current_option_sents = sent_tokenize(text_without_tags, 'russian')
            source_sents[quest_directory] = source_sents[quest_directory].union(current_option_sents)

100%|██████████| 128/128 [29:17<00:00, 13.73s/it]


In [ ]:
with open('source_sents.json', encoding='utf-8') as f:
    source_sents = {key:set(value) for key, value in json.load(f).items()}

In [ ]:
def get_source(sent):
    source = 'new_sent'
    for source_quest in source_sents:
        if sent in source_sents[source_quest]:
            source = source_quest
            break
    return source

In [ ]:
text2source_seq = {}

for file_name in tqdm(os.listdir(current_test_dir)):
    file_path = os.path.join(current_test_dir, file_name)
    with open(file_path, encoding='utf-8') as f:
        text = f.read()
    if 'EOQ' in text:
        quest = text.split('EOQ')[0]
    else:
        quest = text
    text_without_tags = re.sub('BO[QF]|BUTTON|INPUT', '', quest)
    sents = sent_tokenize(text_without_tags, 'russian')
    sources = [get_source(sent) for sent in sents if len(sent.split())>1]
    text2source_seq[file_name] = sources
with open('generated_quests_sources.json', 'w', encoding='utf-8') as f:
    json.dump(text2source_seq, f)

In [ ]:
known_beginnings_original_sents_rates = []
unknown_beginnings_original_sents_rates = []

known_beginnings_original_sents_num_soarces = []
unknown_beginnings_original_sents_num_soarces = []

for quest, source_seq in text2source_seq.items():
    if source_seq:
        num_original_sents = len([source for source in source_seq if source == 'new_sent'])
        original_sents_rate = num_original_sents/len(source_seq)
        num_soarces = len(set([source for source in source_seq if source != 'new_sent']))
        if source_seq[0] == 'new_sent':
            unknown_beginnings_original_sents_rates.append(original_sents_rate)
            unknown_beginnings_original_sents_num_soarces.append(num_soarces)
        else:
            known_beginnings_original_sents_rates.append(original_sents_rate)
            known_beginnings_original_sents_num_soarces.append(num_soarces)

Доля предложений, которых не было в обучающих данных, для квестов с новым и встретившимся в тестовых данных началом

In [ ]:
known_beginnings_original_sents_rates

[0.0, 0.1897810218978102]

In [ ]:
pd.Series(unknown_beginnings_original_sents_rates).describe()

count    133.000000
mean       0.589149
std        0.336207
min        0.002703
25%        0.250000
50%        0.617729
75%        0.909774
max        1.000000
dtype: float64

Количество различных источников из обучающих данных для квестов с новым и встретившимся в тестовых данных началом.
Количество может быть завышено, если предложение встречалось в нескольких квестах

In [ ]:
known_beginnings_original_sents_num_soarces

[4, 9]

In [ ]:
pd.Series(unknown_beginnings_original_sents_num_soarces).describe()

count    133.000000
mean       7.308271
std        6.602548
min        0.000000
25%        2.000000
50%        5.000000
75%       10.000000
max       29.000000
dtype: float64

Посмотрим, какие предложения тригерят начало воспроизведения известного фрагмента и начинает ли после этого воспроизводиться цепочка предложений из известного квеста

In [ ]:
triggers = pd.DataFrame(columns=['trigger', 'known_sent', 'caused_next_known_seq'])


for file_name in tqdm(os.listdir(current_test_dir)):
    file_path = os.path.join(current_test_dir, file_name)
    with open(file_path, encoding='utf-8') as f:
        text = f.read()
    if 'EOQ' in text:
        quest = text.split('EOQ')[0]
    else:
        quest = text
    text_without_tags = re.sub('BO[QF]|BUTTON|INPUT', '', quest)
    sents = sent_tokenize(text_without_tags, 'russian')
    previous_sent_source = None
    no_current_quest_seqs = True
    for i, sent in enumerate(sents):
        current_sent_source = get_source(sent)
        if i and current_sent_source != 'new_sent':
            if previous_sent_source == 'new_sent':
                no_current_quest_seqs = False
                trigger = sents[i-1]
                known_sent = sents[i]
                triggers.loc[len(triggers)] = [trigger, known_sent, False]
                check_seq = True
            elif len(triggers) and current_sent_source==previous_sent_source and not no_current_quest_seqs:
                triggers['caused_next_known_seq'][len(triggers)-1] = True
           
        previous_sent_source = current_sent_source
        

100%|██████████| 135/135 [00:37<00:00,  3.64it/s]


In [ ]:
triggers.to_csv('triggers.csv')

In [ ]:
len(triggers)

1891

In [ ]:
triggers

,trigger,known_sent,caused_next_known_seq
0,"Уже почти стемнело, когда вы с Лосиным...",— Эй!,False
1,Вы с Лосиным братом бежите навстречу незнакомцам.,"Наконец, вы наткнулись на костёр, горящий непо...",False
2,"Однако, несмотря на то, что вокруг было много ...",— Подожди!,False
3,Вы с Лосиным братом подбегаете.,— Где ты был?,False
4,Она участвует в каких-то секретных эксперимент...,— Правда?,False
...,...,...,...
1886,"Хотя он и выглядит полным дурачком, недобрый б...",Дребезжащим голосом старичок говорит: «Славнен...,True
1887,Что Вы будете делать?,.,False
1888,-\tКотенка?,- переспрашиваете Вы.,False
1889,Да\n\nНет\n\nНет\n\n-\tОткуда ты знаешь это?,- спрашиваете Вы.,False


In [ ]:
sequence_triggers = triggers[triggers['caused_next_known_seq']]

In [ ]:
sequence_triggers

,trigger,known_sent,caused_next_known_seq
6,"Вдруг дверь в балаган распахивается, и в балаг...",Иди на СТРАНИЦУ 41.,True
12,"Но когда ты оказываешься достаточно высоко, чт...",Иди на СТРАНИЦУ 34.,True
14,Он наклоняется всё ближе и ближе.,— Нет!,True
17,"Ты чувствуешь, как гниющие губы тянутся к твое...",Иди на СТРАНИЦУ 51.,True
19,У неё начинается головокружение.,Он приближается.,True
...,...,...,...
1880,Деревянный меч со звоном падает на пол. Не дав...,Клинок врага пронзил сердце восставшего орка!,True
1881,[b]Выполни Проверку[/b]\nСложность -2.,Боевые искусства.,True
1883,Начать\n\nНачать\n\nСверкающие глаза шамана ог...,"Знай же, что боги с тобой, и да будешь ты наши...",True
1884,Продолжите путь на север\n\nПовернете на запад...,"Странно, но он внушает доверие.",True


Я ожидала, что триггерами будут универсальные фразы, не привязанные к конкретному сюжету, например, "Что делать дальше?"
Фразы такого рода действительно есть ('Иди на СТРАНИЦУ 39.', 'А что ты помнишь?', 
 '— Может быть, в другой раз.\nОна повесила трубку.'), но также много содерэательных фрагментов. 

 Возможно, сначала генерится что-то приближенное к известному квесту, а потом продолжается он, но это надо проверять

Посмотрим на результаты с большей температурой генерации

In [ ]:
current_test_dir = os.path.join('test_generation', '2.5_temp_3_beams_button_input')

In [ ]:
finished_quests_metrics = pd.DataFrame(columns = ['quest_file', 'total_len', 'num_fragments', 'num_buttons', 'mean_fragment_len'])

for file_name in os.listdir(current_test_dir):
    file_path = os.path.join(current_test_dir, file_name)
    with open(file_path, encoding='utf-8') as f:
        text = f.read()
    if 'EOQ' in text:
        quest = text.split('EOQ')[0]
        total_len = len(quest)
        num_fragments = len(re.findall('BOF', quest))
        num_buttons = len(re.findall('BUTTON', quest))
        mean_fragment_len = count_mean_fragment_len(quest)
        finished_quests_metrics.loc[len(finished_quests_metrics)] = [file_name, total_len, num_fragments, num_buttons, mean_fragment_len]


In [ ]:
for column in ['total_len', 'num_buttons', 'num_fragments', 'mean_fragment_len']:
    finished_quests_metrics[column] = pd.to_numeric(finished_quests_metrics[column])

In [ ]:
finished_quests_metrics['buttons_for_fragment'] = finished_quests_metrics.apply(lambda x: count_mean_num_buttons_for_finished_quest(x.num_fragments, x.num_buttons), axis=1)

In [ ]:
finished_quests_metrics.loc[:, ['total_len', 'num_fragments', 'buttons_for_fragment', 'mean_fragment_len']].describe()

,total_len,num_fragments,buttons_for_fragment,mean_fragment_len
count,134.000000,134.000000,134.000000,133.000000
mean,14840.447761,12.156716,1.727253,989.204070
std,17991.721533,9.405529,0.488533,819.558569
min,147.000000,1.000000,0.000000,42.000000
25%,4971.250000,5.250000,1.465909,631.409091
50%,9476.000000,9.000000,1.666667,795.625000
75%,16735.750000,15.750000,1.933333,1116.285714
max,140387.000000,48.000000,5.000000,6043.173913


In [ ]:
text2source_seq = {}

for file_name in tqdm(os.listdir(current_test_dir)):
    file_path = os.path.join(current_test_dir, file_name)
    with open(file_path, encoding='utf-8') as f:
        text = f.read()
    if 'EOQ' in text:
        quest = text.split('EOQ')[0]
    else:
        quest = text
    text_without_tags = re.sub('BO[QF]|BUTTON|INPUT', '', quest)
    sents = sent_tokenize(text_without_tags, 'russian')
    sources = [get_source(sent) for sent in sents if len(sent.split())>1]
    text2source_seq[file_name] = sources

100%|██████████| 136/136 [00:01<00:00, 78.75it/s]


In [ ]:
known_beginnings_original_sents_rates = []
unknown_beginnings_original_sents_rates = []

known_beginnings_original_sents_num_soarces = []
unknown_beginnings_original_sents_num_soarces = []

for quest, source_seq in text2source_seq.items():
    if source_seq:
        num_original_sents = len([source for source in source_seq if source == 'new_sent'])
        original_sents_rate = num_original_sents/len(source_seq)
        num_soarces = len(set([source for source in source_seq if source != 'new_sent']))
        if source_seq[0] == 'new_sent':
            unknown_beginnings_original_sents_rates.append(original_sents_rate)
            unknown_beginnings_original_sents_num_soarces.append(num_soarces)
        else:
            known_beginnings_original_sents_rates.append(original_sents_rate)
            known_beginnings_original_sents_num_soarces.append(num_soarces)

In [ ]:
known_beginnings_original_sents_rates

[0.794251134644478]

In [ ]:
known_beginnings_original_sents_num_soarces

[9]

In [ ]:
pd.Series(unknown_beginnings_original_sents_rates).describe()

count    135.000000
mean       0.940044
std        0.111144
min        0.067797
25%        0.937779
50%        0.956710
75%        0.979261
max        1.000000
dtype: float64

In [ ]:
pd.Series(unknown_beginnings_original_sents_num_soarces).describe()

count    135.000000
mean       6.733333
std        6.515424
min        0.000000
25%        2.000000
50%        5.000000
75%       10.000000
max       29.000000
dtype: float64

In [ ]:
triggers = pd.DataFrame(columns=['trigger', 'known_sent', 'caused_next_known_seq'])


for file_name in tqdm(os.listdir(current_test_dir)):
    file_path = os.path.join(current_test_dir, file_name)
    with open(file_path, encoding='utf-8') as f:
        text = f.read()
    if 'EOQ' in text:
        quest = text.split('EOQ')[0]
    else:
        quest = text
    text_without_tags = re.sub('BO[QF]|BUTTON|INPUT', '', quest)
    sents = sent_tokenize(text_without_tags, 'russian')
    previous_sent_source = None
    no_current_quest_seqs = True
    for i, sent in enumerate(sents):
        current_sent_source = get_source(sent)
        if i and current_sent_source != 'new_sent':
            if previous_sent_source == 'new_sent':
                no_current_quest_seqs = False
                trigger = sents[i-1]
                known_sent = sents[i]
                triggers.loc[len(triggers)] = [trigger, known_sent, False]
                check_seq = True
            elif len(triggers) and current_sent_source==previous_sent_source and not no_current_quest_seqs:
                triggers['caused_next_known_seq'][len(triggers)-1] = True
           
        previous_sent_source = current_sent_source
        

100%|██████████| 136/136 [00:06<00:00, 19.74it/s]


In [ ]:
triggers

,trigger,known_sent,caused_next_known_seq
0,"А затем слышишь, как двое мужчин со всплесками...",— Смотрите!,False
1,— раздаётся из глубин леса нечеловеческий голос.,Иди на СТРАНИЦУ 8.,True
2,На тебя глядит само воплощение зла — лесной дух!,"— Я вижу, вижу!",False
3,— Ты послан мной спасти мир.\nДух начинает хих...,— Как?,False
4,Да я же пошутил!,— восклицает он.,False
...,...,...,...
1841,"Если ты умрёшь за эту ложь, то останешься жит...",Вот и всё.,False
1842,"Ещё один день...\nТы заметил дверь, которая ве...",Она заперта.,False
1843,Он бесстрастно сказал: «Хорошо.,Следуй за мной».,False
1844,"Если, конечно, ты не сумеешь найти способ побе...",КОНЕЦ,False


In [ ]:
sequence_triggers = triggers[triggers['caused_next_known_seq']]

In [ ]:
sequence_triggers

,trigger,known_sent,caused_next_known_seq
1,— раздаётся из глубин леса нечеловеческий голос.,Иди на СТРАНИЦУ 8.,True
11,"Только когда она поправляет свой передник, ты ...",Это Мелисса!,True
26,"Ты шагаешь в вестибюль, и тут клоунская машина...",— кричит мальчик-ящерица.,True
33,"— Ну, так что тебе от меня нужно?",— спрашивает он.,True
40,— Я выберу его и заберу!,— Нет!,True
...,...,...,...
1834,– полюбопытствовал я.,- Он поправился и не стал оставаться.,True
1835,"Я рассказал ей обо всём, что мне самому удалос...",В лавке что-то забрякало и внимание Джени пере...,True
1836,Получить что-то большее\n\nВзять сумку с деньг...,"Не могу объяснить, почему я сказал именно это.",True
1837,"\n\nВы выбрались на безжизненных скалах, стяну...","Рядом, к счастью пролегала трасса автобана.",True
